# The generator context manager
1. Gen to emit the signal for the contextmanager
2. Done `yield` then the `__enter__` into the context
3. Done running the func which is in the bracket
4. Done `next(gen)` and `__exit__` the context


In [14]:
import sys


# add the generator into the contextmanager
class GeneratorCM(object):

    # initialize using the func(*args, **kwargs)
    def __init__(self, gen):
        self.gen = gen
        print(self.gen)

    # __enter__ call the (func(*args, **kwargs))'s next
    def __enter__(self):
        return next(self.gen)

    def __exit__(self, etype, val, tb):
        try:
            if etype is None:
                next(self.gen)
            else:
                self.gen.throw(etype, val, tb)
            raise RuntimeError("Generator didn't stop")
        except StopIteration:
            return True
        except:
            if sys.exc_info()[1] is not val:
                raise


# CM will receive the func and use 'GeneratorCM' to bracket the func
def contextmanager(func):

    def run(*args, **kwargs):
        return GeneratorCM(func(*args, **kwargs))

    return run


import time


# contextmanager
@contextmanager
def timethis():
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print(f"time: {end - start:.2f}")
        


with timethis():
    n = 1000000
    while n > 0:
        n -= 1

<generator object timethis at 0x00000209DE43D7B0>
time: 0.10


In [ ]:
# you can use random to generate a random number hitter
import random 

def frange(minn, maxn):
    while (x:=random.randint(minn, maxn)) != int(minn + (maxn - minn) // 2 ):
        yield x

[i for i in frange(1, 100)].__len__()

203

In [16]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []

    def __repr__(self):
        return 'Node({!r})'.format(self._value)

    def add_child(self, node):
        self._children.append(node)

    def __iter__(self):
        return iter(self._children)

    def depth_first(self, depth=0):
        yield (depth, self)
        for c in self:
            yield from c.depth_first(depth + 1)

root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for n in root.depth_first():
    print(n)


(0, Node(0))
(1, Node(1))
(2, Node(3))
(2, Node(4))
(1, Node(2))
(2, Node(5))


In [17]:
# a way too complicated version 
# using the depthfirstiterator to manage the iteration
class Node2:
    def __init__(self, value):
        self._value = value
        self._children = []

    def __repr__(self):
        return 'Node({!r})'.format(self._value)

    def add_child(self, node):
        self._children.append(node)

    def __iter__(self):
        return iter(self._children)

    def depth_first(self):
        return DepthFirstIterator(self)


class DepthFirstIterator(object):
    '''
    Depth-first traversal
    '''

    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None

    def __iter__(self):
        return self

    def __next__(self):
        # Return myself if just started; create an iterator for children
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        # If processing a child, return its next item
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        # Advance to the next child and start its iteration
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
        
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for n in root.depth_first():
    print(n)


(0, Node(0))
(1, Node(1))
(2, Node(3))
(2, Node(4))
(1, Node(2))
(2, Node(5))


In [18]:
# define the __reverse__(self) 

class Countdown:
    def __init__(self, start):
        self.start = start

    # Forward iterator
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1

    # Reverse iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1

for rr in reversed(Countdown(30)):
    print(rr)
for rr in Countdown(30):
    print(rr)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [19]:
from typing import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

items = [1, 2, [3, 4, [5, 6], 7], 8]
# Produces 1 2 3 4 5 6 7 8
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


In [ ]:
# easier way to merge two files...
import heapq

a = [1, 4, 7, 10]
b = [2, 5, 6, 11]
list(heapq.merge(a, b))
# with open('sorted_file_1', 'rt') as file1, \
#     open('sorted_file_2', 'rt') as file2, \
#     open('merged_file', 'wt') as outf:

#     for line in heapq.merge(file1, file2):
#         outf.write(line)

[1, 2, 4, 5, 6, 7, 10, 11]

In [ ]:
# iter will iterate the generator until it got the tag as ""
import sys

f = open("/etc/passwd")
for chunk in iter(lambda: f.read(10), ""):
    n = sys.stdout.write(chunk)